In [1]:
#加载需要使用的库
%matplotlib inline
import numpy as np 
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
import pymc3 as pm
#设置seed
np.random.seed(123)

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# 载入数据
data = pd.read_csv("./data/rt_tidy.csv")

In [3]:
#观察数据结构，本数据是从原始数据中抽样出的部分数据
data.head(10)

,Unnamed: 0,X.1,language,user_id,trait,stim_id,order,rt,rating,country,...,ethnicity.x,lab,block,X,Race,Gender,Age,ethnicity.y,gender,log_rt
0,1,606711,SPA,9654,confident,BM-040,41,1903,3,CO,...,NaN,COL_004,2,51,B,M,26.952381,black,male,7.551187
1,2,331376,HU,5381,responsible,WF-015,33,799,7,HU,...,"magyar, kaukázusi",HUN_001,2,93,W,F,24.211111,white,female,6.683361
2,3,162777,ENG,2734,attractive,LF-208,14,1226,1,US,...,Hispanic,USA_038,2,63,L,F,24.642857,latinx,female,7.111512
3,4,614094,SPA,9753,caring,BF-008,67,4561,7,CO,...,ninguno,COL_004,1,35,B,F,24.574713,black,female,8.425297
4,5,732055,PT,11726,aggressive,BF-201,36,912,3,PT,...,Caucasiana,POR_001,2,40,B,F,27.400000,black,female,6.815640
5,6,353796,ENG,5692,attractive,BF-047,105,1446,1,AU,...,Caucasian,AUS_007,1,39,B,F,34.213483,black,female,7.276556
6,7,437062,ENG,6884,mean,WM-256,73,1991,1,CA,...,South Asian,CAN_018,1,120,W,M,30.961538,white,male,7.596392
7,8,88967,ENG,1541,old,BF-004,3,1437,4,US,...,White,USA_113,2,32,B,F,26.983696,black,female,7.270313
8,9,438455,ENG,6897,caring,LM-243,63,2546,6,CA,...,white,CAN_018,2,86,L,M,21.642857,latinx,male,7.842279
9,10,701944,ENG,11134,aggressive,LM-219,13,1629,2,US,...,white,USA_054,1,81,L,M,26.703704,latinx,male,7.395722


### 发现问题：1.存在无关变量，且变量命名存在问题 2.自变量需要清洗出来，需重新编码 3.被试的信息比较混乱，且有大量的缺失值需要处理，大小写也不统一

In [4]:
#针对问题1，重新命名变量
data = data.rename(columns = {'ethnicity.x':'ethnicity_participants','ethnicity.y':'ethnicity_face'})
#利用dropna函数对某一列中的缺失值进行处理，参考：https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
data = data.dropna(subset = ['ethnicity_participants'])
#被试填写的ethnicity比较混乱，大小写不一致，将被试的ethnicity全部变为小写
data['ethnicity_participants'] = data['ethnicity_participants'].str.lower()
#新建一列开始编码自变量
data['independent_variable']=1
#研究白人是否会对白人面孔有更快的反应时，参考这篇博客指出了dataframe的返回规则，并且提出可以使用isin函数: https://zhuanlan.zhihu.com/p/475693888
a=['black']
b=['black']
data.loc[(data['ethnicity_participants'].isin(a)) & (data['ethnicity_face'].isin(b)), 'independent_variable']=0
#利用isin()筛选出我们想要的被试数据： https://www.cnblogs.com/silentteller/p/10871944.html
data_analysis = data[data['ethnicity_participants'].isin(a)]
#删去不需要的列 https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html
data_analysis.drop(['language','user_id','Unnamed: 0','stim_id','order','country','lab','block','X'],axis=1,inplace=True)
#删除rt小于0，大于5000ms的被试
data_analysis=data_analysis.loc[data_analysis['rt']<5000]
data_analysis=data_analysis.loc[data_analysis['rt']>0]

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [10]:
#再次观察数据
data_analysis.head(20)

,trait,rt,rating,sex,age,ethnicity_participants,Race,Gender,Age,ethnicity_face,gender,independent_variable
46,trustworthy,3056,7,f,21.0,black,L,M,33.440000,latinx,male,1
179,sociable,1188,3,f,21.0,black,W,M,19.238636,white,male,1
215,old,1352,6,f,19.0,black,L,M,26.142857,latinx,male,1
398,sociable,1318,4,f,21.0,black,A,F,26.000000,asian,female,1
491,caring,2298,5,m,55.0,black,A,M,26.222222,asian,male,1
513,caring,2203,6,m,23.0,black,B,M,29.225806,black,male,0
654,confident,1984,7,f,20.0,black,B,F,26.457447,black,female,0
878,weird,1343,4,m,22.0,black,L,F,24.642857,latinx,female,1
991,trustworthy,1588,6,m,22.0,black,L,M,27.500000,latinx,male,1
1115,caring,1991,8,m,23.0,black,L,F,24.730769,latinx,female,1


In [11]:
#查看数据信息和类型
data_analysis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 809 entries, 46 to 99259
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   trait                   809 non-null    object 
 1   rt                      809 non-null    int64  
 2   rating                  809 non-null    int64  
 3   sex                     809 non-null    object 
 4   age                     809 non-null    float64
 5   ethnicity_participants  809 non-null    object 
 6   Race                    809 non-null    object 
 7   Gender                  809 non-null    object 
 8   Age                     809 non-null    float64
 9   ethnicity_face          809 non-null    object 
 10  gender                  809 non-null    object 
 11  independent_variable    809 non-null    int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 82.2+ KB


In [12]:
#描述，对同种族和不同种族面孔评分的反应时
data_analysis.groupby('independent_variable').rt.describe()

,count,mean,std,min,25%,50%,75%,max
independent_variable,,,,,,,,
0,205.0,1746.478049,934.743047,19.0,1070.00,1624.0,2279.0,4586.0
1,604.0,1819.132450,991.017807,3.0,1137.75,1734.5,2343.5,4867.0


In [13]:
#查看不同特质下反应时的情况
data_analysis.groupby('trait').rt.describe()

,count,mean,std,min,25%,50%,75%,max
trait,,,,,,,,
aggressive,26.0,744.923077,381.655439,403.0,485.25,543.0,929.00,1786.0
attractive,49.0,2068.142857,1078.492350,730.0,1116.00,2062.0,2515.00,4768.0
caring,86.0,1887.441860,926.956063,379.0,1334.25,1833.5,2308.50,4867.0
confident,73.0,2312.287671,832.314635,751.0,1760.00,2094.0,2675.00,4820.0
dominant,127.0,1875.763780,911.036349,391.0,1137.00,1747.0,2414.00,4422.0
emostable,40.0,1707.250000,1206.649839,186.0,752.50,1524.5,2439.50,4425.0
old,43.0,2149.209302,839.812070,214.0,1478.50,2074.0,2700.50,4066.0
responsible,107.0,1895.691589,805.303065,3.0,1409.00,1717.0,2413.00,4655.0
sociable,104.0,1977.913462,1039.951111,262.0,1172.75,1814.5,2476.75,4707.0


In [14]:
#同\不同种族的反应时分布
data_analysis.groupby(['independent_variable']).rt.plot.density() 

independent_variable
0    AxesSubplot(0.125,0.125;0.775x0.755)
1    AxesSubplot(0.125,0.125;0.775x0.755)
Name: rt, dtype: object

<Figure size 432x288 with 1 Axes>

In [15]:
#直方图
data_analysis.groupby('independent_variable').rt.mean().plot.bar()
plt.show()

<Figure size 432x288 with 1 Axes>

In [16]:
#不同特质的反应时
data_analysis.groupby('trait').rt.mean().plot.bar()
plt.show()

<Figure size 432x288 with 1 Axes>

### 根据数据清理的结果，可以看出种族和特质的类型都对rt数据产生了影响，因此我们希望以特质作为分层的变量，自变量为种族，因变量为rt做广义分层线性模型

In [5]:
# 将数据分层变量trait转换为因子(factor)类型
trait_idxs, trait = pd.factorize(data_analysis.trait)
# 定义学校与数据的映射：即标注哪名学生(行)属于哪一所学校
coords = {
    "trait":
     trait,
    "obs_id": np.arange(len(trait_idxs)),
}

# Normal分布

In [18]:
#建立模型，命名方式为“model_分布方式+类别”
#这里为变截距定斜率的分层模型
#首先定义超先验→由超先验确定先验→引入变量→建立变量关系
with pm.Model(coords=coords) as model_Normal2:
    # 定义效应alpha、beta的超先验 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=1770, sigma=1000)
    sigma_alpha = pm.HalfNormal("sigma_alpha", 100)

    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="trait")
    beta = pm.Normal('beta',mu=100,sd=300)

    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data_analysis.independent_variable, dims="obs_id")
    trait_idx = pm.Data("trait_idx", trait_idxs, dims="obs_id")
    
    # 建立变量间关系，确定p和误差的分布
    mu = pm.Deterministic("mu", alpha[trait_idx] + beta * x)
    sigma1 = pm.HalfNormal("sigma1", 100)

    # 模型的似然
    y = pm.Normal("y", mu=mu, sigma=sigma1, observed=data_analysis.rt, dims="obs_id")


In [19]:
#查看模型图
pm.model_to_graphviz(model_Normal2)

In [20]:
# 检查先验
with model_Normal2:
    prior_checks = pm.sample_prior_predictive(samples=1000)

In [21]:
x = np.random.randint(2, size = 12) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y1 = a + b * x 
    plt.plot(x, y1)

<Figure size 432x288 with 1 Axes>

In [22]:
#查看mu的先验分布
az.plot_density(
    {'mu':prior_checks['mu']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (1000) than draws (809). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 432x288 with 1 Axes>

In [23]:
az.plot_density(
    {
        'mu_alpha':prior_checks['mu_alpha'],
        'sigma_alpha':prior_checks['sigma_alpha'],
        'sigma1':prior_checks['sigma1']
    }
    )
plt.show()

<Figure size 1490.4x331.2 with 3 Axes>

In [24]:
#MCMC采样
with model_Normal2:
    trace_normal2 = pm.sample(draws = 3000, tune= 2800, target_accept=0.9, chains=2, cores= 4,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [sigma1, beta, alpha, sigma_alpha, mu_alpha]


Sampling 2 chains for 2_800 tune and 3_000 draw iterations (5_600 + 6_000 draws total) took 21 seconds.


0, dim: obs_id, 809 =? 809


In [25]:
#采样结果可视化
az.plot_trace(trace_normal2, var_names=['alpha','beta'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>]], dtype=object)

<Figure size 864x288 with 4 Axes>

In [26]:
az.summary(trace_normal2, var_names=['alpha','beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],981.081,107.188,776.365,1180.528,1.591,1.127,4549.0,4556.0,1.0
alpha[1],1938.265,97.114,1761.596,2123.815,1.511,1.069,4122.0,4707.0,1.0
alpha[2],2061.067,130.906,1804.030,2301.486,1.975,1.396,4401.0,4236.0,1.0
alpha[3],1853.116,103.042,1652.451,2042.572,1.645,1.165,3924.0,4244.0,1.0
alpha[4],2233.886,109.975,2033.267,2440.759,1.637,1.158,4515.0,4407.0,1.0
alpha[5],1598.189,146.087,1306.143,1854.673,2.218,1.572,4341.0,4324.0,1.0
alpha[6],1845.343,90.787,1677.620,2014.638,1.509,1.067,3621.0,4355.0,1.0
alpha[7],1690.881,137.712,1440.285,1951.413,1.923,1.360,5126.0,4182.0,1.0
alpha[8],1862.228,95.164,1692.825,2045.398,1.534,1.085,3849.0,4474.0,1.0
alpha[9],1997.769,125.956,1762.855,2231.298,1.926,1.364,4284.0,3946.0,1.0


In [27]:
#计算后验预测分布
with model_Normal2:
    ppc_y2 = pm.sample_posterior_predictive(trace_normal2.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace_normal2, az.from_pymc3(posterior_predictive=ppc_y2), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [28]:
az.plot_ppc(trace_normal2)
plt.show()

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

In [29]:
az.plot_posterior(trace_normal2)

/opt/conda/lib/python3.7/site-packages/arviz/plots/plot_utils.py:496: UserWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (825) in plot_posterior, generating only 40 plots
  UserWarning,


array([[<AxesSubplot:title={'center':'mu_alpha'}>,
        <AxesSubplot:title={'center':'alpha\ntrustworthy'}>,
        <AxesSubplot:title={'center':'alpha\nsociable'}>,
        <AxesSubplot:title={'center':'alpha\nold'}>],
       [<AxesSubplot:title={'center':'alpha\ncaring'}>,
        <AxesSubplot:title={'center':'alpha\nconfident'}>,
        <AxesSubplot:title={'center':'alpha\nweird'}>,
        <AxesSubplot:title={'center':'alpha\ndominant'}>],
       [<AxesSubplot:title={'center':'alpha\nemostable'}>,
        <AxesSubplot:title={'center':'alpha\nresponsible'}>,
        <AxesSubplot:title={'center':'alpha\nattractive'}>,
        <AxesSubplot:title={'center':'alpha\naggressive'}>],
       [<AxesSubplot:title={'center':'alpha\nunhappy'}>,
        <AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'sigma_alpha'}>,
        <AxesSubplot:title={'center':'mu\n0'}>],
       [<AxesSubplot:title={'center':'mu\n1'}>,
        <AxesSubplot:title={'center':'mu\n2'}>,
   

<Figure size 1987.2x3312 with 40 Axes>

In [30]:
#变截距和斜率的分层模型，这里也参考了https://docs.pymc.io/en/v3/pymc-examples/examples/generalized_linear_models/GLM-hierarchical.html
#建立模型，命名方式为“model_分布方式+类别”
#首先定义超先验→由超先验确定先验→引入变量→建立变量关系
with pm.Model(coords=coords) as model_Normal3:
    # 定义效应alpha、beta的超先验 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=1770, sigma=1000) 
    sigma_alpha = pm.HalfNormal("sigma_alpha", 100) 
    mu_beta = pm.Normal("mu_beta", mu=100, sigma=300)
    sigma_beta = pm.HalfNormal("sigma_beta", 100)

    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="trait")
    beta = pm.Normal('beta',mu=mu_beta,sd=sigma_beta, dims="trait")
    
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data_analysis.independent_variable, dims="obs_id")
    trait_idx = pm.Data("trait_idx", trait_idxs, dims="obs_id")
    
    # 建立变量间关系，确定p和误差的分布
    mu = pm.Deterministic("mu", alpha[trait_idx] + beta[trait_idx] * x)
    sigma1 = pm.HalfNormal("sigma1", 100)

    # 模型的似然
    y = pm.Normal("y", mu=mu, sigma=sigma1, observed=data_analysis.rt, dims="obs_id")

In [31]:
pm.model_to_graphviz(model_Normal3)

In [32]:
# 检查先验
with model_Normal3:
    prior_checks = pm.sample_prior_predictive(samples=1000)

In [33]:
x = np.random.randint(2, size = 12) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y1 = a + b * x 
    plt.plot(x, y1)

<Figure size 432x288 with 1 Axes>

In [34]:
#查看mu的先验分布
az.plot_density(
    {'mu':prior_checks['mu']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (1000) than draws (809). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 432x288 with 1 Axes>

In [35]:
az.plot_density(
    {'alpha':prior_checks['alpha'],
    'beta':prior_checks['beta']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (1000) than draws (12). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 993.6x331.2 with 2 Axes>

In [36]:
az.plot_density(
    {
        'mu_alpha':prior_checks['mu_alpha'],
        'sigma_alpha':prior_checks['sigma_alpha'],
        'mu_beta':prior_checks['mu_beta'],
        'sigma_beta':prior_checks['sigma_beta'],
        'sigma1':prior_checks['sigma1']
    }
    )
plt.show()

<Figure size 1490.4x662.4 with 6 Axes>

In [37]:
#MCMC采样
with model_Normal3:
    trace_normal3 = pm.sample(draws = 4000, tune= 3800, target_accept=0.9, chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma1, beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]


Sampling 2 chains for 3_800 tune and 4_000 draw iterations (7_600 + 8_000 draws total) took 50 seconds.
There were 134 divergences after tuning. Increase `target_accept` or reparameterize.
There were 169 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


0, dim: obs_id, 809 =? 809


In [38]:
az.plot_trace(trace_normal3, var_names=['alpha','beta'])
plt.show()

<Figure size 864x288 with 4 Axes>

In [39]:
az.plot_trace(trace_normal3, var_names=['alpha','beta'], combined=True)

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>]], dtype=object)

<Figure size 864x288 with 4 Axes>

In [40]:
az.summary(trace_normal3, var_names=['alpha','beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],1021.507,127.701,785.759,1267.312,2.161,1.528,3461.0,4882.0,1.0
alpha[1],2005.856,122.595,1770.915,2233.402,2.303,1.632,2847.0,4984.0,1.0
alpha[2],2036.651,155.389,1742.827,2335.384,2.859,2.038,2916.0,2261.0,1.0
alpha[3],1756.264,138.239,1488.053,2010.952,2.780,1.966,2471.0,3743.0,1.0
alpha[4],2219.808,128.990,1983.611,2462.438,2.224,1.586,3367.0,2458.0,1.0
alpha[5],1609.193,164.488,1289.919,1906.733,2.470,1.747,4420.0,3779.0,1.0
alpha[6],1793.662,113.835,1574.682,2005.638,1.988,1.406,3282.0,4678.0,1.0
alpha[7],1725.173,154.951,1428.634,2004.167,2.748,1.975,3169.0,5567.0,1.0
alpha[8],1920.004,117.601,1693.710,2134.579,2.096,1.482,3089.0,4497.0,1.0
alpha[9],1917.179,160.196,1633.250,2234.357,3.808,2.779,1741.0,810.0,1.0


In [41]:
#森林图
az.plot_forest(trace_normal3, var_names=['alpha', 'beta'], combined=True);

<Figure size 432x532.8 with 1 Axes>

In [42]:
#计算后验预测分布
with model_Normal3:
    ppc_y = pm.sample_posterior_predictive(trace_normal3.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace_normal3, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)


/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [43]:
az.plot_ppc(trace_normal3)
plt.show()

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

In [44]:
az.plot_posterior(trace_normal3)

/opt/conda/lib/python3.7/site-packages/arviz/plots/plot_utils.py:496: UserWarning: rcParams['plot.max_subplots'] (40) is smaller than the number of variables to plot (838) in plot_posterior, generating only 40 plots
  UserWarning,


array([[<AxesSubplot:title={'center':'mu_alpha'}>,
        <AxesSubplot:title={'center':'mu_beta'}>,
        <AxesSubplot:title={'center':'alpha\ntrustworthy'}>,
        <AxesSubplot:title={'center':'alpha\nsociable'}>],
       [<AxesSubplot:title={'center':'alpha\nold'}>,
        <AxesSubplot:title={'center':'alpha\ncaring'}>,
        <AxesSubplot:title={'center':'alpha\nconfident'}>,
        <AxesSubplot:title={'center':'alpha\nweird'}>],
       [<AxesSubplot:title={'center':'alpha\ndominant'}>,
        <AxesSubplot:title={'center':'alpha\nemostable'}>,
        <AxesSubplot:title={'center':'alpha\nresponsible'}>,
        <AxesSubplot:title={'center':'alpha\nattractive'}>],
       [<AxesSubplot:title={'center':'alpha\naggressive'}>,
        <AxesSubplot:title={'center':'alpha\nunhappy'}>,
        <AxesSubplot:title={'center':'beta\ntrustworthy'}>,
        <AxesSubplot:title={'center':'beta\nsociable'}>],
       [<AxesSubplot:title={'center':'beta\nold'}>,
        <AxesSubplot:title={'

<Figure size 1987.2x3312 with 40 Axes>

# Gamma分布

In [45]:
#Gamma分布，变截距定斜率
with pm.Model(coords=coords) as model_Gamma2:
    # 定义效应alpha的超先验 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=1770, sigma=1000) 
    sigma_alpha = pm.HalfNormal("sigma_alpha", 100) 

    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="trait")
    
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data_analysis.independent_variable, dims="obs_id")
    trait_idx = pm.Data("trait_idx", trait_idxs, dims="obs_id")
    
    # 建立变量间关系
    beta = pm.Normal('beta',mu=0,sd=100)
    mu = pm.Deterministic("mu", alpha[trait_idx] + beta * x)
    sigma1 = pm.HalfNormal("sigma1", 100)

    # 模型的似然
    y = pm.Gamma("y", mu=mu, sigma=sigma1, observed=data_analysis.rt, dims="obs_id")

In [46]:
pm.model_to_graphviz(model_Gamma2)

In [47]:
# 这里会报错，但寻找后似乎是版本问题，Gamma分布抽样中存在数学计算的问题，参考https://discourse.pymc.io/t/domain-error-in-arguments-for-gamma-mlm/4004
#可以继续运行代码，不影响之后的代码
# 检查先验
with model_Gamma2:
    prior_checks = pm.sample_prior_predictive(samples=1000)

ValueError: Domain error in arguments.

In [48]:
x = np.random.randint(2, size = 12) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y1 = a + b * x 
    plt.plot(x, y1)

<Figure size 432x288 with 1 Axes>

In [49]:
#查看mu的先验分布
az.plot_density(
    {
    'mu_alpha':prior_checks['mu_alpha'],
    'sigma_alpha':prior_checks['sigma_alpha'],
    'sigma1':prior_checks['sigma1']},
    )
plt.show()

<Figure size 1490.4x331.2 with 3 Axes>

In [50]:
#MCMC采样
with model_Gamma2:
    trace_Gamma2 = pm.sample(draws = 4000, tune= 3000, target_accept=0.9, chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma1, beta, alpha, sigma_alpha, mu_alpha]


Sampling 2 chains for 3_000 tune and 4_000 draw iterations (6_000 + 8_000 draws total) took 33 seconds.


0, dim: obs_id, 809 =? 809


In [51]:
az.plot_trace(trace_Gamma2, var_names=['alpha','beta'])#分开呈现
plt.show()

<Figure size 864x288 with 4 Axes>

In [52]:
az.plot_trace(trace_Gamma2, var_names=['alpha','beta'], combined=True
)

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>]], dtype=object)

<Figure size 864x288 with 4 Axes>

In [53]:
az.summary(trace_Gamma2, var_names=['alpha','beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],988.538,67.361,860.587,1114.827,1.000,0.709,4541.0,5331.0,1.0
alpha[1],1918.875,84.674,1762.545,2078.674,1.209,0.856,4904.0,5852.0,1.0
alpha[2],2126.393,120.686,1907.209,2357.181,1.347,0.954,8017.0,5859.0,1.0
alpha[3],1884.224,87.365,1714.544,2041.595,1.125,0.795,6026.0,5746.0,1.0
alpha[4],2360.958,99.233,2175.557,2546.325,1.211,0.858,6719.0,5485.0,1.0
alpha[5],1805.115,124.138,1573.125,2035.031,1.406,0.995,7786.0,5936.0,1.0
alpha[6],1928.878,75.298,1794.345,2076.118,1.048,0.742,5171.0,5503.0,1.0
alpha[7],1486.854,112.183,1267.321,1689.836,1.371,0.972,6675.0,5947.0,1.0
alpha[8],1657.647,83.228,1495.131,1806.064,1.242,0.878,4496.0,4752.0,1.0
alpha[9],1983.503,111.104,1776.052,2192.800,1.257,0.890,7803.0,5946.0,1.0


In [54]:
#计算后验预测分布
with model_Gamma2:
    ppc_y = pm.sample_posterior_predictive(trace_Gamma2.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace_Gamma2, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [55]:
az.plot_ppc(trace_Gamma2)
plt.show()

<Figure size 432x288 with 1 Axes>

In [56]:
#变斜率变截距
with pm.Model(coords=coords) as model_Gamma3:
    # 定义效应alpha、beta的超先验 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=1770, sigma=900) 
    sigma_alpha = pm.HalfNormal("sigma_alpha", sigma=100) 
    mu_beta = pm.Normal("mu_beta", mu=100, sigma=300)
    sigma_beta = pm.HalfNormal("sigma_beta", sigma=100)

    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="trait")
    beta = pm.Normal('beta',mu=mu_beta,sd=sigma_beta, dims="trait")

    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data_analysis.independent_variable, dims="obs_id")
    trait_idx = pm.Data("trait_idx", trait_idxs, dims="obs_id")
    
    # 建立变量间关系
    mu = pm.Deterministic("mu", alpha[trait_idx] + beta[trait_idx] * x)
    sigma1 = pm.HalfNormal("sigma1", sigma=100)

    # 模型的似然
    y = pm.Gamma("y", mu=mu, sigma=sigma1, observed=data_analysis.rt, dims="obs_id")

In [57]:
pm.model_to_graphviz(model_Gamma3)

In [58]:
# 检查先验
#同一个问题，GAMMA分布抽样会报错，但不影响后续代码，仍能正常抽样
with model_Gamma3:
    prior_checks = pm.sample_prior_predictive(samples=1000)

ValueError: Domain error in arguments.

In [60]:
x = np.random.randint(2, size = 12) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y1 = a + b * x 
    plt.plot(x, y1)

<Figure size 432x288 with 1 Axes>

In [61]:
#查看mu的先验分布
az.plot_density(
    {'mu':prior_checks['mu'],
     'sigma1':prior_checks['sigma1']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (1000) than draws (809). Passed array should have shape (chains, draws, *shape)
  UserWarning,
/opt/conda/lib/python3.7/site-packages/arviz/stats/density_utils.py:606: UserWarning: Something failed: `x` does not contain any finite number.
  warnings.warn("Something failed: " + str(e))
/opt/conda/lib/python3.7/site-packages/arviz/stats/density_utils.py:606: UserWarning: Something failed: `x` does not contain any finite number.
  warnings.warn("Something failed: " + str(e))


<Figure size 993.6x331.2 with 2 Axes>

In [62]:
az.plot_density(
    {
        'mu_alpha':prior_checks['mu_alpha'],
        'sigma_alpha':prior_checks['sigma_alpha'],
        'mu_beta':prior_checks['mu_beta'],
        'sigma_beta':prior_checks['sigma_beta'],
        'sigma1':prior_checks['sigma1']
    }
    )
plt.show()

<Figure size 1490.4x662.4 with 6 Axes>

In [63]:
with model_Gamma3:
    trace_Gamma3 = pm.sample(draws = 4000, tune= 2000, target_accept=0.9, chains=2, cores= 2,return_inferencedata=True,init="adapt_diag")

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma1, beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]


Sampling 2 chains for 2_000 tune and 4_000 draw iterations (4_000 + 8_000 draws total) took 52 seconds.
There were 2 divergences after tuning. Increase `target_accept` or reparameterize.


0, dim: obs_id, 809 =? 809


In [64]:
az.plot_trace(trace_Gamma3, var_names=['alpha','beta'])#分开呈现
plt.show()

<Figure size 864x288 with 4 Axes>

In [65]:
az.plot_trace(trace_Gamma3, var_names=['alpha','beta'], combined=True)

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>]], dtype=object)

<Figure size 864x288 with 4 Axes>

In [66]:
az.plot_forest(trace_Gamma3, var_names=['alpha', 'beta'], combined=True);

<Figure size 432x532.8 with 1 Axes>

In [67]:
az.summary(trace_Gamma3, var_names=['alpha','beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],1024.529,94.911,845.317,1197.371,1.057,0.750,8090.0,5868.0,1.0
alpha[1],2053.739,133.616,1794.278,2293.241,1.647,1.165,6576.0,6117.0,1.0
alpha[2],2084.294,163.876,1779.409,2395.901,1.874,1.326,7705.0,6100.0,1.0
alpha[3],1698.850,123.556,1477.219,1939.152,1.414,1.000,7650.0,5941.0,1.0
alpha[4],2311.633,137.280,2048.844,2560.872,1.529,1.082,8064.0,6371.0,1.0
alpha[5],1784.655,154.936,1498.370,2078.294,1.753,1.239,7815.0,6311.0,1.0
alpha[6],1776.278,110.075,1567.170,1978.741,1.292,0.914,7260.0,6408.0,1.0
alpha[7],1676.694,173.421,1356.950,2011.105,2.244,1.587,5963.0,5540.0,1.0
alpha[8],1959.526,132.104,1702.362,2199.778,1.878,1.328,4943.0,4424.0,1.0
alpha[9],1879.790,145.550,1597.897,2148.371,1.686,1.192,7458.0,6210.0,1.0


In [68]:
with model_Gamma3:
    #计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace_Gamma3.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace_Gamma3, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [69]:
az.plot_ppc(trace_Gamma3)
plt.show()

<Figure size 432x288 with 1 Axes>

# exGaussian分布

In [6]:
#指数高斯模型
#这里采用了变化斜率变化截距模型
with pm.Model(coords=coords) as model_ExGaussian3:
    m_alpha = pm.Normal("m_alpha", mu=1770, sigma=1000)  
    sig_alpha = pm.HalfNormal("sig_alpha", 100)
    m_beta = pm.Normal("m_beta", mu=100, sigma=300)  
    sig_beta = pm.HalfNormal("sig_beta", 100)
   
    # prior
    alpha = pm.Normal('alpha', mu=m_alpha,sd=sig_alpha, dims="trait")
    beta = pm.Normal('beta',mu=m_beta,sd=sig_beta, dims="trait")
    
    # x
    x = pm.Data("x", data_analysis.independent_variable, dims="obs_id")
    trait_idx = pm.Data("trait_idx", trait_idxs, dims="obs_id")
    mu = pm.Deterministic("mu",  alpha[trait_idx]+ beta[trait_idx]*x) 
    nu = pm.HalfNormal('nu', 100)
    sigma = pm.HalfNormal('sigma', 100)

    y_obs = pm.ExGaussian("y_obs", mu=mu,sigma=sigma,nu=nu, observed=data_analysis.rt, dims="obs_id")

In [7]:
pm.model_to_graphviz(model_ExGaussian3)

In [8]:
# 检查先验
with model_ExGaussian3:
    prior_checks = pm.sample_prior_predictive(samples=1000)

In [9]:
az.plot_density(
    {'mu':prior_checks['mu']}
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (1000) than draws (809). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 432x288 with 1 Axes>

In [10]:
az.plot_density(
    {
        'alpha': prior_checks['alpha']
    }
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (1000) than draws (12). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 432x288 with 1 Axes>

In [11]:
az.plot_density(
    {
        'beta': prior_checks['beta']
    }
    )
plt.show()

/opt/conda/lib/python3.7/site-packages/arviz/data/base.py:174: UserWarning: More chains (1000) than draws (12). Passed array should have shape (chains, draws, *shape)
  UserWarning,


<Figure size 432x288 with 1 Axes>

In [12]:
az.plot_density(
    {'sigma': prior_checks['sigma']
    }
    )
plt.show()

<Figure size 432x288 with 1 Axes>

In [14]:
az.plot_density(
    {'nu': prior_checks['nu']
    }
    )
plt.show()

<Figure size 432x288 with 1 Axes>

In [78]:
x = np.random.randint(2, size = 12) #生成50个假数据，取值为[0,1]

for a, b in zip(prior_checks["alpha"], prior_checks["beta"]):
    y1 = a + b * x 
    plt.plot(x, y1)

<Figure size 432x288 with 1 Axes>

In [79]:
#MCMC采样
with model_ExGaussian3:
    trace_ExGaussian3 = pm.sample(draws = 4000, tune= 2000, target_accept=0.9, chains=2, cores= 2,return_inferencedata=True,init="adapt_diag")

WARNING (theano.tensor.opt): Optimization Warning: The Op erfcx does not provide a C implementation. As well as being potentially slow, this also disables loop fusion.
Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, nu, beta, alpha, sig_beta, m_beta, sig_alpha, m_alpha]


Sampling 2 chains for 2_000 tune and 4_000 draw iterations (4_000 + 8_000 draws total) took 99 seconds.
There were 43 divergences after tuning. Increase `target_accept` or reparameterize.
There were 15 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


0, dim: obs_id, 809 =? 809


In [80]:
az.plot_trace(trace_ExGaussian3, var_names=['alpha','beta'])#分开呈现
plt.show()

<Figure size 864x288 with 4 Axes>

In [81]:
az.plot_trace(trace_ExGaussian3, var_names=['alpha','beta'], combined=True)

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>]], dtype=object)

<Figure size 864x288 with 4 Axes>

In [82]:
az.summary(trace_ExGaussian3, var_names=['alpha','beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],294.630,112.635,82.123,500.894,1.457,1.061,5999.0,5948.0,1.0
alpha[1],1236.836,127.616,996.857,1477.073,2.062,1.458,3825.0,1961.0,1.0
alpha[2],1327.434,149.668,1042.829,1608.208,1.860,1.316,6466.0,5776.0,1.0
alpha[3],964.038,130.929,721.614,1208.455,1.904,1.367,4684.0,3513.0,1.0
alpha[4],1536.613,120.968,1325.635,1778.573,1.564,1.107,5980.0,5923.0,1.0
alpha[5],1002.650,151.055,721.712,1289.945,1.835,1.297,6774.0,5975.0,1.0
alpha[6],988.666,108.844,774.688,1188.596,1.474,1.045,5446.0,5474.0,1.0
alpha[7],863.115,172.509,546.542,1188.421,2.700,1.909,4072.0,2227.0,1.0
alpha[8],1278.925,112.575,1064.318,1490.991,1.746,1.235,4155.0,1989.0,1.0
alpha[9],1086.323,139.478,816.469,1343.639,1.814,1.283,5940.0,5295.0,1.0


In [83]:
with model_ExGaussian3:
    #计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace_ExGaussian3.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace_ExGaussian3, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [84]:
az.plot_ppc(trace_ExGaussian3)
plt.show()

<Figure size 432x288 with 1 Axes>

# Lognormal分布

In [85]:
with pm.Model() as LogNormal:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    sigma = pm.HalfNormal('sigma', sd=100)
    alpha = pm.Normal('alpha', mu=1770, sd=960)
    beta = pm.Normal('beta', mu=100, sd=200)
    # 自变量conf是之前已经载入的数据
    x = pm.Data("x", data_analysis['independent_variable'])
    # 正态分布均值是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta*x) 
    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    # 假定因变量服从log-normal分布
    y_obs = pm.Lognormal('y_obs',mu=mu,sd=sigma,observed=data_analysis['rt'] )

In [86]:
with LogNormal:
    # 不直接使用InferenceData的理由是为了便于之后的模型比较
    trace1_for_comp = pm.sample(draws = 1000, tune=800, target_accept=0.9,chains=2, cores= 4)    
    # 将pymc的采样对象转化为inferencedata
    trace1=az.from_pymc3(trace1_for_comp)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [beta, alpha, sigma]


Sampling 2 chains for 800 tune and 1_000 draw iterations (1_600 + 2_000 draws total) took 7 seconds.


In [87]:
az.plot_trace(trace1,var_names=['alpha','beta','sigma'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

In [88]:
with LogNormal:
     # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace1.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace1, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [89]:
with pm.Model(coords=coords) as model_LogNormal2:
    # 定义效应alpha的超先验 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=1770, sigma=900) 
    sigma_alpha = pm.HalfNormal("sigma_alpha", 100) 

    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="trait")
    beta = pm.Normal('beta',mu=100,sd=300)
    
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data_analysis.independent_variable, dims="obs_id")
    trait_idx = pm.Data("trait_idx", trait_idxs, dims="obs_id")
    
    # 建立变量间关系
    mu = pm.Deterministic("mu", alpha[trait_idx] + beta * x)
    sigma2 = pm.HalfNormal("sigma2", 100)

    # 模型的似然
    y_obs = pm.Lognormal("y_bos", mu=mu, sigma=sigma2, observed=data_analysis.rt, dims="obs_id")

In [90]:
with model_LogNormal2:
    trace_LogNormal2 = pm.sample(draws = 3000, tune=2000, target_accept=0.95,chains=2, cores= 4,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [sigma2, beta, alpha, sigma_alpha, mu_alpha]


Sampling 2 chains for 2_000 tune and 3_000 draw iterations (4_000 + 6_000 draws total) took 61 seconds.


0, dim: obs_id, 809 =? 809


In [91]:
az.plot_trace(trace_LogNormal2, var_names=['alpha','beta'])
#分开呈现
plt.show()

<Figure size 864x288 with 4 Axes>

In [92]:
#计算后验预测分布（PPC)
with model_LogNormal2:
    ppc_y = pm.sample_posterior_predictive(trace_LogNormal2.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace_LogNormal2, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [93]:
#首先定义超先验→由超先验确定先验→引入变量→建立变量关系
with pm.Model(coords=coords) as model_LogNormal3:
    # 定义效应alpha、beta的超先验 Hyperpriors
    mu_alpha = pm.Normal("mu_alpha", mu=1770, sigma=900) 
    sigma_alpha = pm.HalfNormal("sigma_alpha", 100) 
    mu_beta = pm.Normal("mu_beta", mu=100, sigma=300)
    sigma_beta = pm.HalfNormal("sigma_beta", 100)

    # 定义先验
    alpha = pm.Normal('alpha',mu=mu_alpha,sd=sigma_alpha, dims="trait")
    beta = pm.Normal('beta',mu=mu_beta,sd=sigma_beta, dims="trait")
    
    # x为自变量，是之前已经载入的数据
    x = pm.Data("x", data_analysis.independent_variable, dims="obs_id")
    trait_idx = pm.Data("trait_idx", trait_idxs, dims="obs_id")
    
    # 建立变量间关系，确定p和误差的分布
    mu = pm.Deterministic("mu", alpha[trait_idx] + beta[trait_idx] * x)
    sigma2 = pm.HalfNormal("sigma2", 100)

    # 模型的似然
    y_obs = pm.Lognormal("y_obs", mu=mu, sigma=sigma2, observed=data_analysis.rt, dims="obs_id")

In [94]:
with model_LogNormal3:
    trace_LogNormal3 = pm.sample(draws = 3000, tune= 2000, target_accept=0.95, chains=2, cores= 4,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 4 jobs)
NUTS: [sigma2, beta, alpha, sigma_beta, mu_beta, sigma_alpha, mu_alpha]


Sampling 2 chains for 2_000 tune and 3_000 draw iterations (4_000 + 6_000 draws total) took 111 seconds.
There were 185 divergences after tuning. Increase `target_accept` or reparameterize.
The acceptance probability does not match the target. It is 0.86871703916506, but should be close to 0.95. Try to increase the number of tuning steps.
There were 35 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 10% for some parameters.


0, dim: obs_id, 809 =? 809


In [95]:
with model_LogNormal3:
    #计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace_LogNormal3.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace_LogNormal3, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


In [96]:
az.plot_ppc(trace_LogNormal3)
plt.show()

<Figure size 432x288 with 1 Axes>

创建一个空的模型，不包含任何自变量的影响

In [17]:
with pm.Model() as null_model:
    # 定义先验
    mu = pm.Normal("mu", mu=1770, sigma=1000) 
    sigma = pm.HalfNormal("sigma", sigma=100) 
    # 模型的似然
    y = pm.Gamma("y", mu=mu, sigma=sigma, observed=data_analysis.rt)

In [18]:
with null_model:
    trace_null = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, mu]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 5 seconds.


In [99]:
# 将模型的采样结果进行比较
compare_dict = {"model_Normal2": trace_normal2, "model_Normal3": trace_normal3, "model_Gamma3":trace_Gamma3, 
    'model_Gamma2':trace_Gamma2,'null_model':trace_null,'model_ExGaussian3':trace_ExGaussian3,'model_LogNormal3':trace_LogNormal3,
    'LogNormal':trace1,'model_LogNormal2':trace_LogNormal2}
# 选择loo方法进行比较
comp = az.compare(compare_dict, ic='loo')
comp

/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:146: UserWarning: The default method used to estimate the weights for each model,has changed from BB-pseudo-BMA to stacking
  "The default method used to estimate the weights for each model,"
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:656: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  "Estimated shape parameter of Pareto distribution is greater than 0.7 for "
/opt/conda/lib/python3.7/site-packages/arviz/stats/stats.py:656: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is becau

,rank,loo,p_loo,d_loo,weight,se,dse,warning,loo_scale
model_ExGaussian3,0,-6605.734831,18.677134,0.000000,0.352748,25.417973,0.000000,False,log
model_Gamma3,1,-6621.271590,27.571224,15.536759,0.523912,28.690407,12.053226,False,log
model_Gamma2,2,-6630.611715,22.129409,24.876885,0.006773,30.326562,14.528076,True,log
model_Normal3,3,-6660.733710,16.097890,54.998879,0.116568,24.993233,11.344368,False,log
model_Normal2,4,-6661.956260,13.337161,56.221429,0.000000,24.941753,11.430682,False,log
null_model,5,-6741.389634,3.846634,135.654803,0.000000,26.474515,18.729927,False,log
model_LogNormal3,6,-6831.005583,24.533006,225.270752,0.000000,54.449951,45.515018,False,log
model_LogNormal2,7,-6832.835362,21.955631,227.100531,0.000000,54.877420,45.964834,True,log
LogNormal,8,-6915.058908,10.470792,309.324077,0.000000,44.662927,38.519472,False,log


发现先验为指数高斯分布的分层模型表现最好

In [100]:
az.plot_posterior(trace_ExGaussian3, var_names=['beta','alpha'])
plt.show()

<Figure size 1987.2x1987.2 with 24 Axes>

In [101]:
az.plot_forest(trace_ExGaussian3, var_names=['alpha', 'beta'], combined=True);

<Figure size 432x532.8 with 1 Axes>

In [102]:
az.summary(trace_ExGaussian3, var_names=['alpha','beta'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha[0],294.630,112.635,82.123,500.894,1.457,1.061,5999.0,5948.0,1.0
alpha[1],1236.836,127.616,996.857,1477.073,2.062,1.458,3825.0,1961.0,1.0
alpha[2],1327.434,149.668,1042.829,1608.208,1.860,1.316,6466.0,5776.0,1.0
alpha[3],964.038,130.929,721.614,1208.455,1.904,1.367,4684.0,3513.0,1.0
alpha[4],1536.613,120.968,1325.635,1778.573,1.564,1.107,5980.0,5923.0,1.0
alpha[5],1002.650,151.055,721.712,1289.945,1.835,1.297,6774.0,5975.0,1.0
alpha[6],988.666,108.844,774.688,1188.596,1.474,1.045,5446.0,5474.0,1.0
alpha[7],863.115,172.509,546.542,1188.421,2.700,1.909,4072.0,2227.0,1.0
alpha[8],1278.925,112.575,1064.318,1490.991,1.746,1.235,4155.0,1989.0,1.0
alpha[9],1086.323,139.478,816.469,1343.639,1.814,1.283,5940.0,5295.0,1.0
